tf.random.normal(
    shape,
    mean=0.0,
    stddev=1.0,
    dtype=tf.dtypes.float32,
    seed=None,
    name=None
) -> Outputs random values from a normal distribution

tf.nn.softmax(
    logits,
    axis=None,
    name=None,
    dim=None
)

tf.math.reduce_sum(
    input_tensor,
    axis=None,
    keepdims=None,
    name=None,
    reduction_indices=None,
    keep_dims=None
) Computes the sum of elements across dimensions of a tensor. 
Reduces input_tensor along the dimensions given in axis

tf.math.reduce_mean(
    input_tensor,
    axis=None,
    keepdims=None,
    name=None,
    reduction_indices=None,
    keep_dims=None
) Computes the mean of elements across dimensions of a tensor.
Reduces input_tensor along the dimensions given in axis

In [1]:
import tensorflow as tf
import numpy as np
import json

In [21]:
###
### Citest json files si creeaza lista de cuvinte unice din venues. Scrie rezultatul in fisier.
### return numarul de cuvinte.
###
def creeaza_lista_cuvinte_unice():
    lista_unica_cuvinte = []
    
    ## Citeste lista de venues
    r = open('../../Date/Initiale/lista_venue_completa.txt', 'r')
    lista_venues = json.load(r)
    r.close()
    
    ## Creeaza lista de cuvinte unice din venues
    for venue in lista_venues:
        lista_unica_cuvinte += venue
    lista_unica_cuvinte = set(lista_unica_cuvinte)
    
    w = open('../../Date/Initiale/lista_unica_cuvinte_venues.txt','w')
    w.write(json.dumps(list(lista_unica_cuvinte)))
    w.close()

    return len(lista_unica_cuvinte)

###
### Citeste lista de cuvinte unice si creeaza dictionar {cuvant:id}. Scrie rezultatul in fisier.
### param file_lista_cuvinte: calea spre fisierul de unde se citeste lista de cuvinte unice.
### param out_file: calea spre fisierul unde se va scrie rezultatul
###
def creeaza_word2int(file_lista_cuvinte, out_file):
    # Citeste lista cuvinte
    r = open(file_lista_cuvinte,'r')
    lista_unica_cuvinte = json.load(r)
    r.close()
    
    word2int = {}
    
    for i,cuvant in enumerate(lista_unica_cuvinte):
        word2int[cuvant] = i
        
    w = open(out_file,'w')
    w.write(json.dumps(word2int))
    w.close()
    
    
###
### Citeste lista de cuvinte unice si creeaza dictionar {id:cuvant}. Scrie rezultatul in fisier.
### param file_lista_cuvinte: calea spre fisierul de unde se citeste lista de cuvinte unice.
### param out_file: calea spre fisierul unde se va scrie rezultatul
###
def creeaza_int2word(file_lista_cuvinte, out_file):
    # Citeste lista cuvinte
    r = open(file_lista_cuvinte,'r')
    lista_unica_cuvinte = json.load(r)
    r.close()
    
    int2word = {}
    
    for i,cuvant in enumerate(lista_unica_cuvinte):
        int2word[i] = cuvant
        
    w = open(out_file,'w')
    w.write(json.dumps(int2word))
    w.close()

###
### Creeaza un vector de tip one hot.
### param index: pozitia din vector pe care se va pune 1.
### param len_vect: lungimea vectorului.
### return hot_vect: vectorul rezultat.
###
def one_hot(index, len_vect):
    hot_vect = np.zeros(len_vect)
    hot_vect[index] = 1
    return hot_vect

###
###
###
def creeaza_lista_vecini(word2int_file, docs_file, DIM_FEREASTRA, DIM_VOCABULAR):
    # Citeste word2int
    r = open(word2int_file, 'r')
    word2int = json.load(r)
    r.close()
    
    # Citeste lista documente
    r = open(docs_file, 'r')
    lista_doc = json.load(r)
    r.close()
    
    X_train = [] 
    Y_train = [] 
    
    for doc in lista_doc:
        for word_index, word in enumerate(doc):
            for vecin in doc[max(word_index - DIM_FEREASTRA, 0) : min(word_index + DIM_FEREASTRA, len(doc)) + 1] : 
                if vecin != word:
                    X_train.append(one_hot(word2int[ word ], DIM_VOCABULAR))
                    Y_train.append(one_hot(word2int[ vecin ], DIM_VOCABULAR))

    X_train = np.asarray(X_train)
    Y_train = np.asarray(Y_train)
    
    return X_train, Y_train

def log(word2int_file, vectors, file):
    r = open(word2int_file, 'r')
    word2int = json.load(r)
    r.close()
    
    dict_ = {}
    for elem in word2int:
        dict_[elem] = vectors[word2int[elem]].tolist()
    w = open (file, 'w')
    w.write(json.dumps(dict_))
    w.close()       
    

def train ():
    DIM_VOCABULAR = creeaza_lista_cuvinte_unice()
    #creeaza_word2int('../DATE/lista_unica_cuvinte.txt','../DATE/word2int.txt')
    #creeaza_int2word('../DATE/lista_unica_cuvinte.txt','../DATE/int2word.txt')
    
    DIM_FEREASTRA = 1
    X_train, Y_train = creeaza_lista_vecini('../../Date/Initiale/word2int_venues.txt', '../../Date/Initiale/lista_venue_completa.txt', DIM_FEREASTRA, DIM_VOCABULAR)
    
    #### Creare retea
    X = tf.placeholder(tf.float32, shape=(None, DIM_VOCABULAR))
    Y_eticheta = tf.placeholder(tf.float32, shape=(None, DIM_VOCABULAR))

    ## Din layer de input in hidden layer
    EMBEDDING_DIM = 50
    W1 = tf.Variable(tf.random_normal([DIM_VOCABULAR, EMBEDDING_DIM]))
    b1 = tf.Variable(tf.random_normal([EMBEDDING_DIM])) #bias
    layer_hidden = tf.add(tf.matmul(X,W1), b1)

    ## Din hidden layer in output layer
    W2 = tf.Variable(tf.random_normal([EMBEDDING_DIM, DIM_VOCABULAR]))
    b2 = tf.Variable(tf.random_normal([DIM_VOCABULAR]))
    output_layer = tf.nn.softmax(tf.add( tf.matmul(layer_hidden, W2), b2))
    
    
    
    #### Instantiere model tensorflow
    with tf.device('/gpu:0'):
        sess = tf.Session()
        init = tf.global_variables_initializer()
        sess.run(init)

        # Functia de loss -> cross entropy
        cross_entropy_loss = tf.reduce_mean(-tf.reduce_sum(Y_eticheta * tf.log(output_layer), reduction_indices=[1]))
        
        # compute_gradients + apply_gradients = minimize
        learning_rate = 0.02
        train_step = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cross_entropy_loss)

        i = 0
        while(1):
                
            for k in range(DIM_VOCABULAR):
                x_rand = X_train[k].reshape(1,1416) # Transpose to the correct shape
                y_rand = Y_train[k].reshape(1,1416)
            
                sess.run(train_step, feed_dict={X: x_rand, Y_eticheta: y_rand})
                l = sess.run(cross_entropy_loss, feed_dict={X: x_rand, Y_eticheta: y_rand})
                
                #if k % 100 == 0:
                w = open('loss.txt','a')
                w.write(str(l))
                w.write(' ')
                w.close()
                print('loss is : ', l, i)
                i += 1
        
        return vectors

In [22]:
vectors = train()

loss is :  29.307558 0
loss is :  28.620777 1
loss is :  21.221968 2
loss is :  35.593674 3
loss is :  22.57227 4
loss is :  36.124233 5
loss is :  26.996172 6
loss is :  26.63161 7
loss is :  25.143152 8
loss is :  38.19277 9
loss is :  18.638445 10
loss is :  18.116772 11
loss is :  31.524311 12
loss is :  18.569607 13
loss is :  18.518547 14
loss is :  38.32105 15
loss is :  37.033306 16
loss is :  32.90727 17
loss is :  24.200222 18
loss is :  24.01232 19
loss is :  13.317725 20
loss is :  36.7779 21
loss is :  18.750114 22
loss is :  17.602339 23
loss is :  18.557339 24
loss is :  21.574995 25
loss is :  22.614851 26
loss is :  25.81675 27
loss is :  41.625233 28
loss is :  24.645311 29
loss is :  17.197302 30
loss is :  29.531462 31
loss is :  17.396893 32
loss is :  14.28524 33
loss is :  12.078336 34
loss is :  30.963293 35
loss is :  25.262753 36
loss is :  19.622616 37
loss is :  12.834358 38
loss is :  34.70354 39
loss is :  11.7579155 40
loss is :  20.46483 41
loss is :  30

loss is :  28.650816 338
loss is :  11.077621 339
loss is :  19.373404 340
loss is :  18.513004 341
loss is :  23.544537 342
loss is :  6.702019 343
loss is :  22.589348 344
loss is :  17.360794 345
loss is :  23.119991 346
loss is :  23.12943 347
loss is :  19.777365 348
loss is :  19.558714 349
loss is :  11.946439 350
loss is :  17.397486 351
loss is :  15.774638 352
loss is :  15.8556595 353
loss is :  18.413784 354
loss is :  24.00225 355
loss is :  13.468707 356
loss is :  9.9242525 357
loss is :  19.325512 358
loss is :  12.296627 359
loss is :  20.413961 360
loss is :  5.309063 361
loss is :  11.375493 362
loss is :  22.574064 363
loss is :  16.449953 364
loss is :  14.062889 365
loss is :  24.77267 366
loss is :  10.839295 367
loss is :  15.637652 368
loss is :  19.52444 369
loss is :  14.2181225 370
loss is :  1.7722057 371
loss is :  6.4175925 372
loss is :  10.561493 373
loss is :  17.579865 374
loss is :  13.077348 375
loss is :  17.96789 376
loss is :  8.420221 377
loss i

loss is :  2.4794388 670
loss is :  8.055895 671
loss is :  19.70283 672
loss is :  14.710402 673
loss is :  16.298912 674
loss is :  26.82544 675
loss is :  19.083363 676
loss is :  6.5107446 677
loss is :  14.373495 678
loss is :  24.049547 679
loss is :  9.161912 680
loss is :  11.533798 681
loss is :  12.317702 682
loss is :  10.178148 683
loss is :  18.21356 684
loss is :  10.663029 685
loss is :  25.724512 686
loss is :  26.860188 687
loss is :  11.933426 688
loss is :  17.643185 689
loss is :  26.020426 690
loss is :  21.053541 691
loss is :  15.906743 692
loss is :  14.099327 693
loss is :  28.211123 694
loss is :  44.273098 695
loss is :  18.747774 696
loss is :  23.895067 697
loss is :  5.5367413 698
loss is :  0.1418449 699
loss is :  2.7367203 700
loss is :  0.08524682 701
loss is :  18.512817 702
loss is :  12.393153 703
loss is :  11.749721 704
loss is :  15.838948 705
loss is :  19.854141 706
loss is :  24.095486 707
loss is :  15.406483 708
loss is :  21.66388 709
loss 

loss is :  12.541786 1003
loss is :  16.96881 1004
loss is :  12.564292 1005
loss is :  8.515018 1006
loss is :  22.814148 1007
loss is :  26.088104 1008
loss is :  1.1884822 1009
loss is :  20.141224 1010
loss is :  4.0325923 1011
loss is :  9.5317955 1012
loss is :  13.398881 1013
loss is :  20.702806 1014
loss is :  16.130945 1015
loss is :  15.313645 1016
loss is :  5.2337155 1017
loss is :  27.567461 1018
loss is :  6.3038254 1019
loss is :  18.636484 1020
loss is :  20.013409 1021
loss is :  18.692595 1022
loss is :  9.2350025 1023
loss is :  17.091656 1024
loss is :  14.323494 1025
loss is :  12.641488 1026
loss is :  8.14942 1027
loss is :  20.616673 1028
loss is :  9.021041 1029
loss is :  12.884063 1030
loss is :  12.709974 1031
loss is :  17.350916 1032
loss is :  15.825458 1033
loss is :  15.67603 1034
loss is :  15.334635 1035
loss is :  26.524029 1036
loss is :  8.801189 1037
loss is :  25.15092 1038
loss is :  26.725142 1039
loss is :  19.211737 1040
loss is :  20.913456

loss is :  7.9863667 1322
loss is :  18.291431 1323
loss is :  11.796594 1324
loss is :  8.433745 1325
loss is :  13.504492 1326
loss is :  11.300646 1327
loss is :  7.985356 1328
loss is :  9.190207 1329
loss is :  21.121216 1330
loss is :  14.568702 1331
loss is :  22.427975 1332
loss is :  15.711917 1333
loss is :  15.131573 1334
loss is :  19.222301 1335
loss is :  18.353502 1336
loss is :  10.651881 1337
loss is :  13.604344 1338
loss is :  20.929157 1339
loss is :  16.455826 1340
loss is :  27.58824 1341
loss is :  4.2044787 1342
loss is :  24.745335 1343
loss is :  16.11988 1344
loss is :  15.4495535 1345
loss is :  27.006104 1346
loss is :  7.3185577 1347
loss is :  13.698578 1348
loss is :  7.49835 1349
loss is :  20.228582 1350
loss is :  24.590075 1351
loss is :  2.7914517 1352
loss is :  12.046727 1353
loss is :  39.167973 1354
loss is :  6.2879114 1355
loss is :  14.752291 1356
loss is :  21.488384 1357
loss is :  11.683042 1358
loss is :  5.9601574 1359
loss is :  9.34631

...

loss is :  0.23610891 46377
loss is :  0.1873234 46378
loss is :  1.9944386 46379
loss is :  1.04714 46380
loss is :  0.3435196 46381
loss is :  0.11133378 46382
loss is :  0.60411525 46383
loss is :  0.03798037 46384
loss is :  0.12719594 46385
loss is :  0.14412941 46386
loss is :  2.6097221 46387
loss is :  1.7155259 46388
loss is :  0.059480015 46389
loss is :  0.029777706 46390
loss is :  1.0770208 46391
loss is :  0.0132398 46392
loss is :  0.2040756 46393
loss is :  0.15156494 46394
loss is :  0.09796276 46395
loss is :  0.100274846 46396
loss is :  0.41915038 46397
loss is :  0.13804871 46398
loss is :  1.1397132 46399
loss is :  0.083799236 46400
loss is :  0.8824461 46401
loss is :  0.20961843 46402
loss is :  0.034200985 46403
loss is :  0.32876542 46404
loss is :  0.2028862 46405
loss is :  0.33748636 46406
loss is :  0.47553927 46407
loss is :  0.05752815 46408
loss is :  0.8121716 46409
loss is :  0.041919157 46410
loss is :  0.344772 46411
loss is :  0.53553456 46412
los

loss is :  0.5049515 46680
loss is :  0.050639313 46681
loss is :  1.2366683 46682
loss is :  0.16907836 46683
loss is :  1.1982923 46684
loss is :  0.2185073 46685
loss is :  0.3521816 46686
loss is :  0.10588295 46687
loss is :  0.48895776 46688
loss is :  0.047435302 46689
loss is :  0.5281523 46690
loss is :  1.7491841 46691
loss is :  1.3207437 46692
loss is :  1.1063426 46693
loss is :  0.14074649 46694
loss is :  0.9614872 46695
loss is :  0.75882894 46696
loss is :  0.5047255 46697
loss is :  1.9541123 46698
loss is :  0.728205 46699
loss is :  0.40834752 46700
loss is :  1.0729231 46701
loss is :  0.77394307 46702
loss is :  0.1643319 46703
loss is :  0.15627323 46704
loss is :  0.08101524 46705
loss is :  0.069706075 46706
loss is :  0.0077685085 46707
loss is :  0.49485233 46708
loss is :  0.6948026 46709
loss is :  1.9657621 46710
loss is :  1.5617479 46711
loss is :  2.754352 46712
loss is :  1.8567928 46713
loss is :  0.29546863 46714
loss is :  0.14332885 46715
loss is :

loss is :  2.2488744 46984
loss is :  0.4080902 46985
loss is :  0.1470088 46986
loss is :  0.93996143 46987
loss is :  2.4398515 46988
loss is :  0.32023227 46989
loss is :  0.24729833 46990
loss is :  0.9712318 46991
loss is :  2.6362913 46992
loss is :  0.2169064 46993
loss is :  0.17508043 46994
loss is :  0.63447547 46995
loss is :  2.2675426 46996
loss is :  0.34025708 46997
loss is :  0.22392206 46998
loss is :  0.68517625 46999
loss is :  1.2670141 47000
loss is :  0.40581742 47001
loss is :  0.2976015 47002
loss is :  0.579272 47003
loss is :  2.9258838 47004
loss is :  0.7925334 47005
loss is :  0.37791407 47006
loss is :  0.5184134 47007
loss is :  2.2657404 47008
loss is :  0.5949419 47009
loss is :  1.7244948 47010
loss is :  0.983433 47011
loss is :  0.4353676 47012
loss is :  1.1186348 47013
loss is :  0.93053794 47014
loss is :  0.13802421 47015
loss is :  1.4856339 47016
loss is :  0.8326244 47017
loss is :  0.96473634 47018
loss is :  0.2375664 47019
loss is :  2.5268

loss is :  1.963133 47286
loss is :  0.2631342 47287
loss is :  0.20000012 47288
loss is :  0.09157004 47289
loss is :  0.06535552 47290
loss is :  0.26128265 47291
loss is :  0.17984813 47292
loss is :  0.21957943 47293
loss is :  2.1503384 47294
loss is :  0.11897578 47295
loss is :  0.11468305 47296
loss is :  0.062017262 47297
loss is :  2.3913825 47298
loss is :  0.24933805 47299
loss is :  0.18728897 47300
loss is :  0.33337897 47301
loss is :  1.6792924 47302
loss is :  0.11716128 47303
loss is :  2.0257 47304
loss is :  0.5125737 47305
loss is :  1.6299033 47306
loss is :  0.2276867 47307
loss is :  0.19219384 47308
loss is :  2.0048575 47309
loss is :  2.9450758 47310
loss is :  0.6023123 47311
loss is :  2.2940068 47312
loss is :  0.86305624 47313
loss is :  0.71577156 47314
loss is :  1.4505453 47315
loss is :  1.5981536 47316
loss is :  0.19378224 47317
loss is :  0.70702404 47318
loss is :  0.28625736 47319
loss is :  0.292787 47320
loss is :  1.2832325 47321
loss is :  0.

KeyboardInterrupt: 